**ALGORITMO 4 BUSQUEDA MENOR F(n)**

In [ ]:


# Importaciones y configuración

import random
import math
from colorama import Fore, Style, init

init(autoreset=True)


In [ ]:


# Funciones auxiliares


def generar_tablero(filas, columnas, num_celdas_cerradas):
    tablero = [[random.randint(1, filas*columnas) for _ in range(columnas)] for _ in range(filas)]
    cerradas = set()
    while len(cerradas) < num_celdas_cerradas:
        x, y = random.randint(0, columnas-1), random.randint(0, filas-1)  
        cerradas.add((x, y))
    for (x, y) in cerradas:
        tablero[y][x] = "X"  
    return tablero, cerradas


def imprimir_tablero(tablero, inicio, meta, camino=None):
    filas, columnas = len(tablero), len(tablero[0])
    for y in range(filas):
        fila = []
        for x in range(columnas):
            if (x, y) == inicio:
                fila.append(Fore.BLUE + "S" + Style.RESET_ALL)
            elif (x, y) == meta:
                fila.append(Fore.YELLOW + "M" + Style.RESET_ALL)
            elif camino and (x, y) in camino:
                fila.append(Fore.GREEN + "*" + Style.RESET_ALL)
            elif tablero[y][x] == "X":
                fila.append(Fore.RED + "X" + Style.RESET_ALL)
            else:
                fila.append(str(tablero[y][x]))
        print("\t".join(fila))
    print()


def heuristica(a, b):
    return math.sqrt((b[0]-a[0])**2 + (b[1]-a[1])**2)


def vecinos(filas, columnas, pos):
    direcciones = [(-1,0),(1,0),(0,-1),(0,1), (-1,-1),(-1,1),(1,-1),(1,1)]
    for dx, dy in direcciones:
        x2, y2 = pos[0]+dx, pos[1]+dy
        if 0 <= x2 < columnas and 0 <= y2 < filas:
            yield (x2, y2)


In [ ]:


#Algoritmo principal de búsqueda


def buscar_camino(
    tablero,
    inicio,
    meta,
    permitir_diagonales=True,
    evitar_cortes_esquina=False,
    mostrar_minitablero=True, 
    parar_si_meta_vecina=True,
):
    filas, cols = len(tablero), len(tablero[0])

    def es_bloqueada(p):
        return tablero[p[1]][p[0]] == "X"

    def vecinos_local(p):
        x, y = p
        for v in vecinos(filas, cols, p):
            if not permitir_diagonales:
                if v[0] != x and v[1] != y:
                    continue
            if evitar_cortes_esquina and (v[0] != x and v[1] != y):
                if tablero[y][v[0]] == "X" and tablero[v[1]][x] == "X":
                    continue
            yield v

    def ordenar_vecinos(p):
        cand = []
        for v in vecinos_local(p):
            if es_bloqueada(v):
                continue
            g = tablero[v[1]][v[0]]
            h = heuristica(v, meta)
            cand.append((g + h, h, g, v))
        cand.sort(key=lambda t: (t[0], t[1], t[2]))
        return [v for _, _, _, v in cand]

    pila = [(inicio, ordenar_vecinos(inicio))]
    en_camino = {inicio}

    def pintar_estado(titulo):
        if not mostrar_minitablero:
            return
        print(titulo)
        camino_actual = [n for n, _ in pila]
        imprimir_tablero(tablero, inicio, meta, camino_actual)

    print(f"\nEstoy en  {inicio}:")
    pintar_estado("Tablero (inicio):")

    while pila:
        actual, cand = pila[-1]
        if actual == meta:
            return [n for n, _ in pila]

        while cand and cand[0] in en_camino:
            cand.pop(0)

        if parar_si_meta_vecina and meta in cand:
            peso = tablero[meta[1]][meta[0]]
            h = 0.0
            print(f"  Vecino {meta} con peso={peso}")
            print(f"    g({meta}) = {peso}")
            print(f"    h({meta}) = sqrt(({meta[0]}-{meta[0]})²+({meta[1]}-{meta[1]})²) = {h:.2f}")
            print(f"    f({meta}) = g+h = {peso}+{h:.2f} = {peso+h:.2f}")
            en_camino.add(meta)
            pila.append((meta, []))
            pintar_estado("Tablero tras avanzar (meta vecina):")
            print(f"\nEstoy en  {meta}:")
            continue

        if not cand:
            salgo, _ = pila.pop()
            en_camino.remove(salgo)
            if pila:
                print(f"Regresar: {salgo} -> {pila[-1][0]}")
                pintar_estado("Tablero tras regresar:")
                print(f"\nEstoy en  {pila[-1][0]}:")
            continue

        siguiente = cand.pop(0)
        peso = tablero[siguiente[1]][siguiente[0]]
        h = heuristica(siguiente, meta)
        print(f"  Vecino {siguiente} con peso={peso}")
        print(f"    g({siguiente}) = {peso}")
        print(f"    h({siguiente}) = sqrt(({siguiente[0]}-{meta[0]})²+({siguiente[1]}-{meta[1]})²) = {h:.2f}")
        print(f"    f({siguiente}) = g+h = {peso}+{h:.2f} = {peso+h:.2f}")

        en_camino.add(siguiente)
        pila.append((siguiente, ordenar_vecinos(siguiente)))
        pintar_estado("Tablero tras avanzar:")
        print(f"\nEstoy en  {siguiente}:")
    return None


In [ ]:

#  Ejecución principal

filas = int(input("Número de filas (eje Y): "))
columnas = int(input("Número de columnas (eje X): "))
num_cerradas = int(input("Número de celdas cerradas: "))

tablero, cerradas = generar_tablero(filas, columnas, num_cerradas)

print("\n=== Tablero inicial ===")
imprimir_tablero(tablero, (-1,-1), (-1,-1))  

sx, sy = map(int, input("Coordenadas de salida (x y): ").split())  
mx, my = map(int, input("Coordenadas de meta (x y): ").split())

while tablero[my][mx] == "X":
    print("La meta no puede ser una celda cerrada. Elige otra.")
    mx, my = map(int, input("Coordenadas de meta (x y): ").split())

inicio = (sx, sy)
meta = (mx, my)

camino = buscar_camino(tablero, inicio, meta)

print("\n=== Resultado ===")
if camino:
    imprimir_tablero(tablero, inicio, meta, camino)
    print("Camino encontrado:", camino)
else:
    print("No existe camino posible")


Número de filas (eje Y):  5
Número de columnas (eje X):  5
Número de celdas cerradas:  5



=== Tablero inicial ===
3	23	X	13	24
20	21	19	25	8
14	2	13	X	X
X	7	13	15	13
X	9	10	24	14



Coordenadas de salida (x y):  0 4
Coordenadas de meta (x y):  0 0



Estoy en  (0, 4):
Tablero (inicio):
M	23	X	13	24
20	21	19	25	8
14	2	13	X	X
X	7	13	15	13
S	9	10	24	14

  Vecino (1, 3) con peso=7
    g((1, 3)) = 7
    h((1, 3)) = sqrt((1-0)²+(3-0)²) = 3.16
    f((1, 3)) = g+h = 7+3.16 = 10.16
Tablero tras avanzar:
M	23	X	13	24
20	21	19	25	8
14	2	13	X	X
X	*	13	15	13
S	9	10	24	14


Estoy en  (1, 3):
  Vecino (1, 2) con peso=2
    g((1, 2)) = 2
    h((1, 2)) = sqrt((1-0)²+(2-0)²) = 2.24
    f((1, 2)) = g+h = 2+2.24 = 4.24
Tablero tras avanzar:
M	23	X	13	24
20	21	19	25	8
14	*	13	X	X
X	*	13	15	13
S	9	10	24	14


Estoy en  (1, 2):
  Vecino (2, 2) con peso=13
    g((2, 2)) = 13
    h((2, 2)) = sqrt((2-0)²+(2-0)²) = 2.83
    f((2, 2)) = g+h = 13+2.83 = 15.83
Tablero tras avanzar:
M	23	X	13	24
20	21	19	25	8
14	*	*	X	X
X	*	13	15	13
S	9	10	24	14


Estoy en  (2, 2):
  Vecino (2, 3) con peso=13
    g((2, 3)) = 13
    h((2, 3)) = sqrt((2-0)²+(3-0)²) = 3.61
    f((2, 3)) = g+h = 13+3.61 = 16.61
Tablero tras avanzar:
M	23	X	13	24
20	21	19	25	8
14	*	*	X	X
X	*	*	15	13


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




M	23	X	13	24
20	21	19	25	8
14	*	*	X	X
X	*	*	*	13
S	9	10	*	*


Estoy en  (4, 4):
  Vecino (4, 3) con peso=13
    g((4, 3)) = 13
    h((4, 3)) = sqrt((4-0)²+(3-0)²) = 5.00
    f((4, 3)) = g+h = 13+5.00 = 18.00
Tablero tras avanzar:
M	23	X	13	24
20	21	19	25	8
14	*	*	X	X
X	*	*	*	*
S	9	10	*	*


Estoy en  (4, 3):
Regresar: (4, 3) -> (4, 4)
Tablero tras regresar:
M	23	X	13	24
20	21	19	25	8
14	*	*	X	X
X	*	*	*	13
S	9	10	*	*


Estoy en  (4, 4):
Regresar: (4, 4) -> (3, 3)
Tablero tras regresar:
M	23	X	13	24
20	21	19	25	8
14	*	*	X	X
X	*	*	*	13
S	9	10	*	14


Estoy en  (3, 3):
Regresar: (3, 3) -> (3, 4)
Tablero tras regresar:
M	23	X	13	24
20	21	19	25	8
14	*	*	X	X
X	*	*	15	13
S	9	10	*	14


Estoy en  (3, 4):
  Vecino (4, 4) con peso=14
    g((4, 4)) = 14
    h((4, 4)) = sqrt((4-0)²+(4-0)²) = 5.66
    f((4, 4)) = g+h = 14+5.66 = 19.66
Tablero tras avanzar:
M	23	X	13	24
20	21	19	25	8
14	*	*	X	X
X	*	*	15	13
S	9	10	*	*


Estoy en  (4, 4):
  Vecino (4, 3) con peso=13
    g((4, 3)) = 13
    h((4, 3)) = sqr

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Tablero tras regresar:
M	23	X	*	*
20	21	*	25	8
14	*	*	X	X
X	*	13	15	13
S	9	10	24	14


Estoy en  (4, 0):
  Vecino (3, 1) con peso=25
    g((3, 1)) = 25
    h((3, 1)) = sqrt((3-0)²+(1-0)²) = 3.16
    f((3, 1)) = g+h = 25+3.16 = 28.16
Tablero tras avanzar:
M	23	X	*	*
20	21	*	*	8
14	*	*	X	X
X	*	13	15	13
S	9	10	24	14


Estoy en  (3, 1):
  Vecino (4, 1) con peso=8
    g((4, 1)) = 8
    h((4, 1)) = sqrt((4-0)²+(1-0)²) = 4.12
    f((4, 1)) = g+h = 8+4.12 = 12.12
Tablero tras avanzar:
M	23	X	*	*
20	21	*	*	*
14	*	*	X	X
X	*	13	15	13
S	9	10	24	14


Estoy en  (4, 1):
Regresar: (4, 1) -> (3, 1)
Tablero tras regresar:
M	23	X	*	*
20	21	*	*	8
14	*	*	X	X
X	*	13	15	13
S	9	10	24	14


Estoy en  (3, 1):
Regresar: (3, 1) -> (4, 0)
Tablero tras regresar:
M	23	X	*	*
20	21	*	25	8
14	*	*	X	X
X	*	13	15	13
S	9	10	24	14


Estoy en  (4, 0):
Regresar: (4, 0) -> (3, 0)
Tablero tras regresar:
M	23	X	*	24
20	21	*	25	8
14	*	*	X	X
X	*	13	15	13
S	9	10	24	14


Estoy en  (3, 0):
  Vecino (3, 1) con peso=25
    g((3, 1)) = 25